In [265]:
import requests
import random
from bs4 import BeautifulSoup


In [273]:
url = 'https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0061.TW'
user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "accept-encoding": "gzip, deflate, br", 
    "accept-language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7", 
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "sec-gpc": "1",
    "referer": "https://www.google.com/",
    "dnt": "1", 
    "upgrade-Insecure-Requests": "1", 
    "user-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Mobile Safari/537.36", 
}



In [26]:
# test requests but no use
res = requests.get(url, headers=headers).content
soup = BeautifulSoup(res, 'html.parser')
# soup

In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from time import sleep
import numpy as np
from tqdm import tqdm


In [6]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15'
chrome_path = 'C:/Users/Administrator/.wdm/drivers/chromedriver/win32/92.0.4515.107/chromedriver.exe'
url = 'https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=2824.HK'


### ETFs
- https://www.moneydj.com/etf/x/Basic/Basic0007.xdjhtm?etfid=SPY

In [7]:
import json

In [28]:
# catch all etf company names
opt = webdriver.ChromeOptions()
opt.add_argument('--user-agent=%s' % user_agent)
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)

driver.get(url)
sleep(0.5)

etf_list = [item.text for item in driver.find_elements_by_xpath('//select[@id="Menu1"]//option')]
print(etf_list)

driver.quit()



['2ndVote Funds', 'AAM', 'abrdn', 'Absolute Investment Advisers', 'Acquirers Funds', 'ACV ETFs', 'Adaptive Investments', 'Adasina', 'AdvisorShares', 'AGFiQ', 'Alexis Practical Tactical', 'Alger', 'AllianzIM', 'Alpha Architect', 'AlphaClone', 'AlphaMark Advisors', 'ALPS Advisors', 'Alternative Access Funds', 'AltShares', 'American Century Investments', 'Amplifyetfs', 'Apollo Funds', 'Applied Finance Funds', 'Aptus Capital Advisors', 'ARK ETF Trust', 'Arrow Investment Advisors', 'ArrowShares', 'ASA Gold and Precious Metals', 'ASYMshare', 'Avantis Investors', 'Ballast Asset Management', 'Barclays Bank', 'Beyond Investing', 'Blue Horizon Capital', 'BMO Capital Markets', 'BNY Mellon', 'Cabana', 'Cabot ETF Partners', 'Cambria Funds', 'Capital Link', 'Change Finance', 'Changebridge', 'Charles Schwab', 'ClearShares', 'Columbia Threadneedle Investments', 'Core Alternative Capital', 'Cornercap', 'Counterpoint Mutual Funds', 'Credit Suisse', 'CrossingBridge', 'Davis ETFs', 'Day Hagan', 'db X-trac

In [17]:
etf_list_asia = etf_list[212:]


In [59]:
# catch all etf of each companies
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)

etf_options = dict()
driver.get(url)
for etf in tqdm(etf_list_asia):
    
#     idx = etf_list.index(etf)
#     print(idx)

    # get current company's etf list
    option_list = [item.text for item in driver.find_elements_by_xpath('//select[@id="Menu2"]//option')]
    etf_options.update({etf: option_list})
    
    # go to another company
    WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, 
                                        '//span[@id="DivMenu1"]'))
    ).click()
    action = ActionChains(driver)
    action.send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    
    sleep(np.random.randint(1, 5))


driver.quit()




100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [04:49<00:00,  4.67s/it]


In [76]:
# catch all stock (element ratio) of each etf of each companies
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)

etf_element = dict()
for etf in tqdm(etf_options.keys()):
    for etfid in etf_options[etf]:
        
        etfid = etfid.split('〈')[1][:-1]
        url = f'https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid={etfid}'
        driver.get(url)
        
        try:
            table = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, 
                                                '//table[@id="ctl00_ctl00_MainContent_MainContent_stable3"]')))
            table_content = table.text
        except:
            print(url)
            table_content = ' '
#         driver.find_element_by_xpath('//table[@id="ctl00_ctl00_MainContent_MainContent_stable3"]').text
        etf_element.update({etfid: table_content})
        
        sleep(np.random.randint(1, 5))

driver.quit()
        

 10%|███████████████████▎                                                                                                                                                                                    | 6/62 [01:53<21:44, 23.30s/it]

https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00753L.TW


 11%|██████████████████████▌                                                                                                                                                                                 | 7/62 [03:22<39:25, 43.02s/it]

https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00638R.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00648R.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00647L.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00632R.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00631L.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00681R.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00680L.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00738U.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00673R.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00642U.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00674R.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00708L.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00635U.TW
https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=00707R.TW
https://www.moneydj.com/ETF/X/Basi

 13%|█████████████████████████▍                                                                                                                                                                           | 8/62 [07:42<1:37:31, 108.37s/it]

...

 89%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                      | 55/62 [34:38<04:00, 34.36s/it]

https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=020009.TW


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                   | 56/62 [34:46<02:38, 26.43s/it]

https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=2840.HK


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [36:34<00:00, 35.39s/it]


In [92]:
# reformat the table content to list of tuples
etf_element_new = dict()
for etfid in etf_element.keys():
    
    if etf_element[etfid] == ' ':
        etf_element_new.update({etfid: ' '})
    else:
        list_table = [line.split(' ') for line in etf_element[etfid].split('\n')]
        etf_element_new.update({etfid: [(row[0], row[1], row[2]) for row in list_table[1:]]})



In [5]:
# write to json
with open('etf_element_ratio.json', 'w', encoding='utf-8') as f:
    json.dump(etf_element_new, f, ensure_ascii=False, indent=4)
    

In [98]:
# load from json
with open('etf_element_ratio.json', 'r', encoding='utf-8') as f:
    etf_element_test = json.load(f)
    

In [112]:
# A mapping dict between ETF id & name
etf_id_name_mapping = dict()

for comp in etf_options.keys():
    for etf in etf_options[comp]:
        etf_id = etf.split('〈')[-1][:-1]
        etf_name = etf.split('〈')[0]
        etf_id_name_mapping.update({etf_id: etf_name})
        
# etf_id_name_mapping


### 基金
- https://www.moneydj.com/funddj/yb/YP301000.djhtm

In [9]:
import re
import pandas as pd


In [104]:
url = 'https://www.moneydj.com/funddj/yb/YP301000.djhtm'
account = 'abaoyang'
password = 'YGyang615thebest'


In [126]:
# catch type of funds
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)
driver.get(url)
sleep(1)
fund_types = [(item.text, item.get_attribute('href')) for item 
              in driver.find_elements_by_xpath('//div[@class="InternalSearch"]//a')]
print(fund_types[:5])

fund_company_list = list()
for item in tqdm(fund_types):
    driver.get(item[1])
    fund_company = [(item.text, item.get_attribute('href')) for item 
                         in driver.find_elements_by_xpath('//table[@id="oMainTable"]//td/a') if len(item.text.strip()) > 0]
    print(fund_company[:6])
    fund_list = fund_company[0::2]
    print(fund_list[:3])
    company_list = fund_company[1::2]
    company_list = [item[0] for item in company_list]
    print(company_list[:3])
    for i in range(len(fund_list)):
        fund_company_list += [(company_list[i], fund_list[i][0], fund_list[i][1])]
    
    sleep(np.random.randint(3, 7))
    
    
driver.quit()




  0%|                                                                                                                                                                                                                | 0/32 [00:00<?, ?it/s]

[('指數型', 'https://www.moneydj.com/funddj/yb/YP302000.djhtm?a=ET000001'), ('科技類', 'https://www.moneydj.com/funddj/yb/YP302000.djhtm?a=ET001001'), ('中小型', 'https://www.moneydj.com/funddj/yb/YP302000.djhtm?a=ET001004'), ('一般股票型', 'https://www.moneydj.com/funddj/yb/YP302000.djhtm?a=ET001005'), ('中概股型', 'https://www.moneydj.com/funddj/yb/YP302000.djhtm?a=ET001006')]
[('元大富櫃50基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACKY39'), ('元大投信', 'https://www.moneydj.com/funddj/ya/yp020000.djhtm?a=BFZYTA'), ('元大台灣金融基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=AC0055'), ('元大投信', 'https://www.moneydj.com/funddj/ya/yp020000.djhtm?a=BFZYTA'), ('元大台灣中型100基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=AC0051'), ('元大投信', 'https://www.moneydj.com/funddj/ya/yp020000.djhtm?a=BFZYTA')]
[('元大富櫃50基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACKY39'), ('元大台灣金融基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=AC0055'), ('元大台灣中型100基金', 'https://www.moneydj.com/fu

  3%|██████▎                                                                                                                                                                                                 | 1/32 [00:06<03:11,  6.16s/it]

[('安聯台灣科技基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACDD04'), ('安聯投信', 'https://www.moneydj.com/funddj/ya/yp020000.djhtm?a=BFZ002'), ('新光創新科技基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACTS09'), ('新光投信', 'https://www.moneydj.com/funddj/ya/yp020000.djhtm?a=BFZTSA'), ('兆豐國際電子基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACII11'), ('兆豐投信', 'https://www.moneydj.com/funddj/ya/yp020000.djhtm?a=BFZIIA')]
[('安聯台灣科技基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACDD04'), ('新光創新科技基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACTS09'), ('兆豐國際電子基金', 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACII11')]
['安聯投信', '新光投信', '兆豐投信']


  6%|████████████▌                                                                                                                                                                                           | 2/32 [00:13<03:18,  6.62s/it]

...

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=t


  9%|██████████████████                                                                                                                                                                                | 338/3632 [54:32<9:03:31,  9.90s/it]

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=t


  9%|██████████████████                                                                                                                                                                                | 339/3632 [54:40<8:51:10,  9.68s/it]


KeyboardInterrupt: 

In [139]:
# catch ratio of funds
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)
fund_url = 'https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACFP72'
driver.get(fund_url)
sleep(15)


fund_ratio = dict()
fund_id_name_mapping = dict()

for fund in tqdm(fund_company_list):
    
    fund_name = fund[1]
    fund_id = fund[2].split('a=')[-1]
    fund_id_name_mapping.update({fund_id: fund_name})
    
    fund_url = f'https://www.moneydj.com/funddj/yl/yp013000.djhtm?a={fund_id}'
    driver.get(fund_url)
    try:
        ratio_content = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//table[@class="t04"]//table[@class="t01"]/tbody')))
        ratio_content = ratio_content.text
    except:
        print(fund_url)
        ratio_content = ' '
    
    finally:
        fund_ratio.update({fund_id: ratio_content})
        
    sleep(np.random.randint(3, 7))


driver.quit()




  0%|▏                                                                                                                                                                                                   | 4/3632 [00:23<6:19:42,  6.28s/it]

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACCA98


  0%|▎                                                                                                                                                                                                   | 6/3632 [00:38<6:59:17,  6.94s/it]

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACTR12


  0%|▍                                                                                                                                                                                                   | 7/3632 [00:48<7:46:16,  7.72s/it]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 3630/3632 [6:46:04<00:17,  8.65s/it]

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACJS79


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 3631/3632 [6:46:15<00:09,  9.35s/it]

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACFH64


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3632/3632 [6:46:27<00:00,  6.71s/it]


In [143]:
fund_company_list[30]

('富邦投信',
 '富邦臺灣加權單日反向一倍基金',
 'https://www.moneydj.com/funddj/ya/yp010000.djhtm?a=ACFP72')

In [304]:
# store to json file
with open('fund_element_ratio_tables.json', 'w', encoding='utf-8') as f:
    json.dump(fund_ratio, f, ensure_ascii=False, indent=4)
    
with open('fund_id_name_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(fund_id_name_mapping, f, ensure_ascii=False, indent=4)


In [302]:
# 只存股票項目不存比例
fund_ratio_simple = dict()
for k in fund_ratio.keys():
    
    content = ' '.join(fund_ratio[k].split('\n')[3:])
    cl = np.unique([char.strip() for char in re.findall(r'[\u4e00-\u9fff- a-zA-Z]+|91APP', content) 
                    if (char.strip() != '') and (char.strip() != '-') 
                    and (char.strip() != 'KY') and (char.strip() != '-KY')]).tolist()
    
    fund_ratio_simple.update({k: cl})


# store to json file
with open('fund_holdings.json', 'w', encoding='utf-8') as f:
    json.dump(fund_ratio_simple, f, ensure_ascii=False, indent=4)


In [312]:
# 轉存 3 元組為 csv
collect = list()
for k in tqdm(fund_ratio_simple.keys()):
    for item in fund_ratio_simple[k]:
        collect += [(k, '持有', item)]

len(collect)
pd.DataFrame(collect, columns=['fund_id', 'rel', 'target_name']).drop_duplicates().to_csv('fund_holding_relation.csv', index=False)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3632/3632 [00:00<00:00, 301872.86it/s]


### 公司基本資訊
- MoneyDJ: https://concords.moneydj.com/z/zc/zca/zca.djhtm?a=2303
    - 有些網頁是亂碼、有些會有空格問題抓起來比較麻煩
    
- Goodinfo: https://goodinfo.tw/StockInfo/BasicInfo.asp?STOCK_ID=2303


In [104]:
url = 'https://concords.moneydj.com/z/zc/zca/zca.djhtm?a=2303'

list_stock_id = pd.read_csv('listed_company.csv', header=None)
list_stock_id.columns = ['comp']
list_stock_id = list_stock_id['comp'].str.split('　', expand=True)
list_stock_id.columns = ['id', 'name']
stock_ids = list_stock_id.id.unique()
stock_ids


array(['1101', '1102', '1103', '1104', '1108', '1109', '1110', '1201',
       '1203', '1210', '1213', '1215', '1216', '1217', '1218', '1219',
       '1220', '1225', '1227', '1229', '1231', '1232', '1233', '1234',
       '1235', '1236', '1256', '1301', '1303', '1304', '1305', '1307',
       '1308', '1309', '1310', '1312', '1313', '1314', '1315', '1316',
       '1319', '1321', '1323', '1324', '1325', '1326', '1337', '1338',
       '1339', '1340', '1341', '1342', '1402', '1409', '1410', '1413',
       '1414', '1416', '1417', '1418', '1419', '1423', '1432', '1434',
       '1435', '1436', '1437', '1438', '1439', '1440', '1441', '1442',
       '1443', '1444', '1445', '1446', '1447', '1449', '1451', '1452',
       '1453', '1454', '1455', '1456', '1457', '1459', '1460', '1463',
       '1464', '1465', '1466', '1467', '1468', '1470', '1471', '1472',
       '1473', '1474', '1475', '1476', '1477', '1503', '1504', '1506',
       '1507', '1512', '1513', '1514', '1515', '1516', '1517', '1519',
      

In [ ]:
url = f'https://concords.moneydj.com/z/zc/zca/zca.djhtm?a={stock_id}'
res = requests.get(url, headers=headers).content
soup = BeautifulSoup(res, 'html.parser')
keys = [item.text for item in soup.find_all('td', {'class': 't4t1'})]
values = [item.text for item in soup.find_all('td', {'class': ['t3n1', 't3t1']}) if item.text.strip() != '']
try:
    idx = keys.index('公司地址')
    _dict = dict(zip(keys[:idx+1], values[:idx+1]))

except:
    print(f'broken id: {stock_id}')

finally:
    print(_dict)
    

In [189]:
def getCompanyTable(url):
    print(url)
    res = requests.get(url, headers=headers).content
    soup = BeautifulSoup(res, 'html.parser')
    keys = [item.text for item in soup.find_all('td', {'class': 't4t1'})]
    values = [item.text for item in soup.find_all('td', {'class': ['t3n1', 't3t1']}) if item.text.strip() != '']
    try:
        idx = keys.index('公司地址')
        _dict = dict(zip(keys[:idx+1], values[:idx+1]))
    except:
        print(f'broken ')
        print(keys, values)
        _dict = ''
    return _dict




import chardet
chardet.detect('¶}½L»ù'.encode('utf-8'))


{'encoding': 'utf-8', 'confidence': 0.938125, 'language': ''}

In [190]:
# for stock_id in broken_ids:
stock_id = 1324
url = f'https://concords.moneydj.com/z/zc/zca/zca.djhtm?a={stock_id}'
getCompanyTable(url)


https://concords.moneydj.com/z/zc/zca/zca.djhtm?a=1324


{'開盤價': '10.95',
 '最高價': '11.2',
 '最低價': '10.95',
 '收盤價': '11.15',
 '漲跌': '+0.15',
 '一年內最高價': '14.2',
 '一年內最低價': '9.8',
 '本益比': 'N/A',
 '一年內最大量': '3,424',
 '一年內最低量': '1',
 '成交量': '119',
 '同業平均本益比': '11.93',
 '一年來最高本益比': 'N/A',
 '一年來最低本益比': '44.59',
 '盤後量': '0',
 '殖利率': '2.69%',
 '總市值': '838',
 '85年來最高總市值': '2,967',
 '85年來最低總市值': '488',
 '今年以來': '9.14%',
 '每股淨值(元)': '12.50',
 '營業毛利率': '13.38%',
 '貝他值': '0.64',
 '最近一週': '2.29%',
 '每人營收(仟元)': '761.00',
 '營業利益率': '-0.98%',
 '標準差': '1.83%',
 '最近一個月': '0.90%',
 '每股營收(元)': '3.68',
 '稅前淨利率': '-0.90%',
 '最近二個月': '0.90%',
 '負債比例': '23.04%',
 '資產報酬率': '-0.28%',
 '最近三個月': '4.69%',
 '股價淨值比': '0.89',
 '股東權益報酬率': '-0.37%',
 '股本(億, 台幣)': '7.51',
 '現金股利(元)': '0.3',
 '成立時間': '53/12/08',
 '股票股利': '0',
 '初次上市(櫃)日期': '86/12/26',
 '盈餘配股': '0',
 '股務代理': '永豐金證02-23816288',
 '公積配股': '0',
 '董事長': '鈞鼎投資股份有限公司/張德照',
 '總經理': '黃林慶',
 '發言人': '張雅琪',
 '營收比重': 'PVC膠帶100.00% (2020年)',
 '公司電話': '02-25066666',
 '網址': 'http://www.globetape.com.tw/',
 '公司地址': '台北市中山區南京東路三段6

In [174]:
# comp_dict = dict()
# broken_ids = list()

selected_stock_ids = ['1236','1307','1321','1419','1439','1470','1541','1592','1614','1727','1805','2028','2115','2115','2211','2250','2388','2390','2404','2405','2424','2430','2433','2438','2468','2501','2501','2543','2712','2753','2910','2936','2945','3021','3035','3051','3059','3138','3266','3419','3515','3532','3701','3714','4190','4426','4440','4555','4560','4755','4930','4994','5222','5225','5284','5546','5608','6136','6172','6189','6201','6209','6224','6230','6289','6405','6442','6515','6715','6743','6754','6756','6768','6770','6781','6790','6806','8104','8427','8467','8478','8482','8488','9926','9927']

for stock_id in tqdm(selected_stock_ids):

    url = f'https://concords.moneydj.com/z/zc/zca/zca.djhtm?a={stock_id}'
    res = requests.get(url, headers=headers).content
    soup = BeautifulSoup(res, 'html.parser')
    keys = [item.text for item in soup.find_all('td', {'class': 't4t1'})]
    values = [item.text for item in soup.find_all('td', {'class': ['t3n1', 't3t1']}) if item.text.strip() != '']
    try:
        idx = keys.index('公司地址')
        _dict = dict(zip(keys[:idx+1], values[:idx+1]))
        comp_dict.update({stock_id: _dict})
    
    except:
        print(f'broken id: {stock_id}')
        broken_ids.append(stock_id)
        continue
    
    finally:
        sleep(np.random.randint(3, 8))



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [07:12<00:00,  5.09s/it]


In [175]:
for stock_id in tqdm(selected_stock_ids):
    print(comp_dict[stock_id])
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 33918.36it/s]

{'開盤價': '24.5', '最高價': '25.55', '最低價': '24.2', '收盤價': '25.55', '漲跌': '+0.05', '一年內最高價': '25.7', '一年內最低價': '13', '本益比': 'N/A', '一年內最大量': '4,818', '一年內最低量': '2', '成交量': '2,318', '同業平均本益比': '22.49', '一年來最高本益比': 'N/A', '一年來最低本益比': '33.64', '盤後量': '12', '殖利率': '0.78%', '總市值': '2,768', '85年來最高總市值': '3,158', '85年來最低總市值': '299', '今年以來': '82.25%', '每股淨值(元)': '14.26', '營業毛利率': '23.71%', '貝他值': '0.39', '最近一週': '21.38%', '每人營收(仟元)': 'N/A', '營業利益率': '-4.62%', '標準差': '2.71%', '最近一個月': '80.57%', '每股營收(元)': '3.73', '稅前淨利率': '-3.51%', '最近二個月': '83.81%', '負債比例': '40.45%', '資產報酬率': '-0.47%', '最近三個月': '83.15%', '股價淨值比': '1.79', '股東權益報酬率': '-0.87%', '股本(億, 台幣)': '10.83', '現金股利(元)': '0.2', '成立時間': '65/06/14', '股票股利': '0', '初次上市(櫃)日期': '90/09/17', '盈餘配股': '0', '股務代理': '群益金鼎證02-27023999', '公積配股': '0', '董事長': '張云綺', '總經理': '張云綺', '發言人': '余彩雲', '營收比重': '威化巧克力37.05%、糕(囍)餅(盒)27.74%、乳加巧克力21.03%、其他14.19% (2020年)', '公司電話': '02-29180786', '網址': 'http://www.hunya.com.tw', '公司地址': '新北市新店區寶興路45巷8弄3號5樓'}
{'開盤價': '20.85',

In [177]:
selected_stock_ids[2]

'1321'

In [194]:
# write to json
with open('company_basic_info.json', 'w', encoding='utf-8') as f:
    json.dump(comp_dict, f, ensure_ascii=False, indent=4)
    

#### 類股分類
- 上市
- 上櫃
- 興櫃
- 電子產業
- 概念股
- 集團股


In [274]:
def getStockCates(idx):
    
    market_categories = ['上市','上櫃','興櫃','電子產業','概念股','集團股']
    market_cat = market_categories[idx]
    url = f'https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT={market_cat}'
    headers['User-Agent'] = random.choice(user_agent_list)
    res = requests.get(url, headers=headers).content
    soup = BeautifulSoup(res, 'html.parser')
    list_stock_cate = soup.find('table', {'id': f'MENU{idx+2}'}).find_all('td', {'colspan': '4'})
    list_stock_cate = [re.findall(r'[\u4e00-\u9fff]+', str(item)) for item in list_stock_cate]
    list_stock_cate = [item[-1] for item in list_stock_cate if len(item) >0 and market_cat not in item[-1]]
    return list_stock_cate


list_stock_cate = getStockCates(idx=1)
list_stock_cate


AttributeError: 'NoneType' object has no attribute 'find_all'

In [210]:
industries = list_stock_cate.copy()
url = f'https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT=上市&INDUSTRY_CAT={list_stock_cate[0]}'


In [275]:
def getCateStockList(market_cat, ind):
    
    url = f'https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT={market_cat}&INDUSTRY_CAT={ind}'
    headers['User-Agent'] = random.choice(user_agent_list)
    res = requests.get(url, headers=headers).content
    soup = BeautifulSoup(res, 'html.parser')
    list_stock = [item.text for item in soup.find('table', {'class': 'r10_0_0_10 b1 p4_1'}).find_all('td')]
    list_stock = [(x, y) for x, y in zip(list_stock[::2], list_stock[1::2]) if x != '代號']
    return list_stock



In [263]:
market_categories = ['上市','上櫃','興櫃','電子產業','概念股','集團股']
stock_categories = dict()
sleep_time = 10

for idx in tqdm(range(len(market_categories))):
    _ind_stocks = dict()
    market_cat = market_categories[idx]
    industries = getStockCates(idx=idx)
    print(industries)
    
    sleep(np.random.randint(5, 8))
    
    for ind in industries:
        try:
            list_stock = getCateStockList(market_cat=market_cat, ind=ind)
        except:
            print(url)
            # 發生錯誤時 多休息幾秒緩衝 避免被網站擋
            sleep_time += 5
            continue
            
        finally:
            _ind_stocks.update({ind: list_stock})
            print(ind, _ind_stocks[ind], '\n')

        sleep(np.random.randint(sleep_time, sleep_time+5))
        
    stock_categories.update({market_cat: _ind_stocks})
    print(market_cat, stock_categories[market_cat])



  0%|                                                                                                                                                                                                                 | 0/6 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'find_all'

In [260]:
stock_categories.keys()
stock_categories['上櫃']['保險業']


[('5878', '台名')]

In [262]:
with open('stock_categories.json', 'w', encoding='utf-8') as f:
    json.dump(stock_categories, f, ensure_ascii=False, indent=4)


In [ ]:
url = f'https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID={stock_id}'


#### SemEval 2016 Task5 ABSA datasets
- xml parser

In [12]:
from xml.etree import ElementTree
import xmltodict

# trial data
url = 'https://alt.qcri.org/semeval2016/task5/data/uploads/trial-data/mobilephones_trial_chinese.xml'

res = requests.get(url, headers=headers).content
# tree = ElementTree.fromstring(res)
res_dict = xmltodict.parse(res)
res_dict


OrderedDict([('Reviews',
              OrderedDict([('Review',
                            [OrderedDict([('@rid', '1'),
                                          ('sentences',
                                           OrderedDict([('sentence',
                                                         [OrderedDict([('@id',
                                                                        '1:0'),
                                                                       ('text',
                                                                        '用了一个月的台电C280客观评价，')]),
                                                          OrderedDict([('@id',
                                                                        '1:1'),
                                                                       ('text',
                                                                        '优点：音质非常不错，'),
                                                                       ('Opinions',
          

In [41]:
res_dict.keys()
res_dict['Reviews'].keys()
len(res_dict['Reviews']['Review'])
res_dict['Reviews']['Review'][0].keys()
res_dict['Reviews']['Review'][0]['sentences'].keys()
len(res_dict['Reviews']['Review'][0]['sentences']['sentence'])
res_dict['Reviews']['Review'][0]['sentences']['sentence'][1].keys()
res_dict['Reviews']['Review'][0]['sentences']['sentence'][1]['Opinions']['Opinion']['@category']


'MULTIMEDIA DEVICES#QUALITY'

#### Other

In [179]:
# check table data
# 英文公司名中間有空格，和分隔符衝突

empty = list()
diff = list()
valid = list()
for k in fund_ratio.keys():
    
    cl = list()
    for row in fund_ratio[k].split('\n'):
        cl.append([char.strip() for char in row.split(' ') if len(char.strip()) > 0])
    
    try: 
        check = len(cl[3])
        if check not in [6, 8]:
            diff.append(k)
        else:
            valid.append(k)
    except:
        empty.append(k)


empty
diff

['ACFP71',
 'ACCY98',
 'ACFP72',
 'ACML07',
 'ACCP10',
 'ACII21',
 'ACPS01',
 'ACFT121',
 'ACFT122',
 'ACFH67',
 'ACFH68',
 'ACFT119',
 'ACFT120',
 'ACFT116',
 'ACFT115',
 'ACFT117',
 'ACFT118',
 'ACJS61',
 'ACJS62',
 'ACJS66',
 'ACJS65',
 'ACPS60',
 'ACPS58',
 'ACPS59',
 'ACJS26',
 'ACNC100',
 'ACNC98',
 'ACNC101',
 'ACNC99',
 'ACCB46',
 'ACCB47',
 'ACFH22',
 'ACFH56',
 'ACCY81',
 'ACDD15',
 'ACJF92',
 'ACJF146',
 'ACYC17',
 'ACFH23',
 'ACII02',
 'ACII14',
 'ACNC56',
 'ACDD17',
 'ACNC88',
 'ACNC57',
 'ACNC89',
 'ACAP136',
 'ACNC141',
 'ACNC139',
 'ACNC142',
 'ACFH26',
 'ACNC140',
 'ACAP137',
 'ACTS82',
 'ACTS84',
 'ACTS83',
 'ACKH46',
 'ACFT84',
 'ACFT83',
 'ACKH45',
 'ACKG90',
 'ACKG89',
 'ACKG119',
 'ACCS16',
 'ACKG81',
 'ACKG37',
 'ACCA176',
 'ACCA103',
 'ACCA104',
 'ACCA179',
 'ACKG38',
 'ACKG82',
 'ACAP135',
 'ACKH175',
 'ACGC23',
 'ACAP142',
 'ACIC1001',
 'ACIC10',
 'ACCA101',
 'ACCA178',
 'ACCA102',
 'ACCA181',
 'ACCA100',
 'ACCA180',
 'ACCA177',
 'ACCA99',
 'ACAP71',
 'ACFT81'

In [108]:
# test the login pop-up
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)
driver.get('https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACKY39')
sleep(1)

# 在網頁外面的彈出 selenium 沒辦法操作
#action = ActionChains(driver)
#action.send_keys(account).send_keys(Keys.TAB).send_keys(password).send_keys(Keys.ENTER).perform()


In [138]:
driver.quit()

In [218]:
cl = list()
for row in fund_ratio[diff[11]].split('\n'):
    cl.append([char.strip() for char in row.split(' ') if len(char.strip()) > 0])

cl
# [row.split(' ') for row in fund_ratio['AC0001'].split('\n')]
diff[11]

'ACFT119'

In [300]:
k = 'ACAI175'

content = ' '.join(fund_ratio[k].split('\n')[3:])
cl = np.unique([char.strip() for char in re.findall(r'[\u4e00-\u9fff- a-zA-Z]+|91APP', content) 
                if (char.strip() != '') and (char.strip() != '-')]).tolist()

content

'C 0   7.02 1.19% ALL 5.1   3.20 0.04% BAC 6.45 K*   5.87 0.46% RGA 6.2   2.47 0.13% AEGON 5.1   4.83 0.08% SRE 5.75   2.43 0.04% ATH 6.35 A   3.44 0.04% HIG 7.875   2.25 0.23% T 5.35   3.34 0.00 CTL 6.75   1.95 0.01%'

In [313]:
len(fund_ratio_simple.keys())

3632

In [301]:
fund_ratio[k]

'股票名稱 持股\n(千股) 比例 增減 股票名稱 持股\n(千股) 比例 增減\nC 0   7.02 1.19% ALL 5.1   3.20 0.04%\nBAC 6.45 K*   5.87 0.46% RGA 6.2   2.47 0.13%\nAEGON 5.1   4.83 0.08% SRE 5.75   2.43 0.04%\nATH 6.35 A   3.44 0.04% HIG 7.875   2.25 0.23%\nT 5.35   3.34 0.00 CTL 6.75   1.95 0.01%'

In [327]:
url = 'https://www.tej.com.tw/webtej/doc/uid.htm'

driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)
driver.get(url)
text = [item.text for item in driver.find_elements_by_xpath('//font')]
sleep(1)
driver.quit()

In [347]:
end = text.index('上櫃公司代碼一覽表')
start = text.index('1101   台泥')
stock_comp = [item.strip() for item in text[start:end] if (item.strip() != '') and re.findall('[0-9]+', item)]
stock_names = [item.split(' ')[-1].strip() for item in stock_comp]
stock_ids = [item.split(' ')[0].strip() for item in stock_comp]


In [335]:
re.findall('[0-9]+', '1110   東泥')

['1110']

In [352]:
df_stock_comp = pd.DataFrame([stock_ids, stock_names]).T
df_stock_comp.columns = ['id', 'name']
df_stock_comp.to_csv('df_stock_comp.csv', index=False)

In [353]:
df_stock_comp

,id,name
0,1101,台泥
1,1102,亞泥
2,1103,嘉泥
3,1104,環泥
4,1107,建台
...,...,...
642,9941,裕融
643,9942,茂順
644,9943,好樂迪
645,9944,新麗
